In [1]:
from scipy.io import wavfile
import scipy.signal as sps
import os
import librosa
import pandas as pd
import numpy as np
import json

import tensorflow as tf

gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

2022-11-20 11:08:30.948423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 11:08:31.103696: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-20 11:08:31.644991: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mt/miniconda3/envs/tf/lib/:/home/mt/miniconda3/lib/:/home/mt/miniconda3/envs/tf/lib/
2022-11-20 11:08:31.645083: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic librar

In [2]:
def resample(path,new_rate = 16000):
    sampling_rate, data = wavfile.read(path)
    samples = round(len(data) * float(new_rate) / sampling_rate)
    new_data = sps.resample(data, samples)
    return new_data

In [3]:
def js_to_np(path,max_len=50):    
    f = open(path)
    data = json.load(f)
    t=[[j['e'] for j in i['l']] for i in data]
    t=[j for i in t for j in i]
    if (len(t)>50): t=t[:50]
    return np.pad(t,(0,50-len(t)))

In [4]:
def inp():
    ds=["data/train/songs/"+f for f in os.listdir("data/train/songs/") if os.path.isfile(os.path.join("data/train/songs/", f))]
    for i in ds:
        label=i.replace("songs","labels").replace("wav","json")
        t=np.mean(resample(i),axis=1)
        yield (librosa.feature.melspectrogram(y=np.pad(t,(0,480000-len(t))),sr=16000),js_to_np(label))
x=[]
y=[]
for i in inp():
    x.append(i[0])
    y.append(i[1])
x=np.array(x)
y=np.array(y)
with open('test.npy', 'wb') as f:
    np.save(f, x)
    np.save(f, y)

In [5]:
with open('test.npy', 'rb') as f:
    x = np.load(f)
    y = np.load(f)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1028)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1028,activation = 'relu'),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dense(256,activation = 'relu'),
    tf.keras.layers.Dense(128,activation = 'relu'),
    tf.keras.layers.Dense(50,activation = 'relu')
])
model.compile(loss="mse",optimizer='adam',metrics=["mae"])
model.fit(x=x,y=y, batch_size=10, epochs=50)

2022-11-20 11:13:48.613947: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 11:13:48.618565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-20 11:13:48.618909: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-20 11:13:48.619071: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/50


2022-11-20 11:13:55.085976: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


106/106 [==============================] - 15s 90ms/step - loss: 53514652.0000 - mae: 5151.3364
Epoch 2/50
106/106 [==============================] - 9s 88ms/step - loss: 43939008.0000 - mae: 4619.7915
Epoch 3/50
106/106 [==============================] - 9s 88ms/step - loss: 43664892.0000 - mae: 4582.7642
Epoch 4/50
106/106 [==============================] - 9s 89ms/step - loss: 41398244.0000 - mae: 4355.2168
Epoch 5/50
106/106 [==============================] - 9s 89ms/step - loss: 40332992.0000 - mae: 4194.2832
Epoch 6/50
106/106 [==============================] - 9s 89ms/step - loss: 39182584.0000 - mae: 4084.6235
Epoch 7/50
106/106 [==============================] - 10s 93ms/step - loss: 37492696.0000 - mae: 3915.0310
Epoch 8/50
106/106 [==============================] - 10s 93ms/step - loss: 35862952.0000 - mae: 3720.9771
Epoch 9/50
106/106 [==============================] - 10s 93ms/step - loss: 34608996.0000 - mae: 3572.4543
Epoch 10/50
106/106 [==============================] 

In [125]:
def np_to_js(pred,path):
    f = open(path)
    data = json.load(f)
    count=0
    pre=0
    pre_c=0
    for i in data:
        for j in i['l']:
            j['s']=pre
            j['e']=int(round(pred[0][count])) if count < 50 else 0
            pre=int(round(pred[0][count])) if count < 50 else 0
            count=count+1
        i['e']=pre
        i['s']=pre_c
        pre_c=pre        
    with open(path, "w",encoding='utf8') as outfile:
        json.dump(data, outfile, ensure_ascii=False)

In [126]:
def out():
    ds=["data/public_test/songs/"+f for f in os.listdir("data/public_test/songs/") if os.path.isfile(os.path.join("data/public_test/songs/", f))]
    for i in ds:
        label=i.replace("songs","lyrics/new_labels_json").replace("wav","json")
        t=np.mean(resample(i),axis=1)
        t=librosa.feature.melspectrogram(y=np.pad(t,(0,480000-len(t))),sr=16000)
        pred = model.predict(tf.expand_dims(t,0))
        np_to_js(pred,label)

In [127]:
out()

1/1 [==============================] - 0s 21ms/step
